In [ ]:
import pandas as pd
import os
import re
from wordcloud import WordCloud
import gensim
from gensim.utils import simple_preprocess
import nltk
import ssl
import gensim.corpora as corpora
from pprint import pprint
import pyLDAvis.gensim_models as gensimvis
import pickle
import pyLDAvis
import csv


In [ ]:
os.chdir('..')

papers = pd.read_csv('Volumes/CIRCUIT_HD2/CRIMSON/Topic_Modeling_Practice/papers.csv')
print(papers.head())

In [ ]:
#remove punctuation
papers['paper_text_processed']= \
papers['paper_text'].map(lambda x: re.sub('[,\.!?]', '', x))

#convert titles to lowercase
papers['paper_text_processed']= \
papers['paper_text_processed'].map(lambda x: x.lower())

In [ ]:
long_string = ','.join(list(papers['paper_text_processed'].values))

wordcloud = WordCloud(background_color='white',max_words=5000, contour_width=3, contour_color='steelblue')

wordcloud.generate(long_string)

wordcloud.to_image()

In [ ]:
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')
from nltk.corpus import stopwords

#tokenization (also removing stopwords)
stop_words = stopwords.words('english')
stop_words.extend(['from','subject','re','edu','use'])

def sent_to_words(sentences):
    for sentence in sentences:
        yield((gensim.utils.simple_preprocess(str(sentence),deacc = True)))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
            if word not in stop_words] for doc in texts]

data = papers.paper_text_processed.values.tolist()
data_words = list(sent_to_words(data))

data_words = remove_stopwords(data_words)

In [ ]:
id2word = corpora.Dictionary(data_words)

texts = data_words

corpus = [id2word.doc2bow(text) for text in texts]

print(corpus[:1][0][:30])

In [ ]:
num_topics = 10

lda_model = gensim.models.LdaMulticore(corpus=corpus, id2word=id2word, num_topics=num_topics)

#doc_lda = lda_model[corpus]

pprint(lda_model.print_topics())

In [ ]:
def get_top_n(dict_elem, n):
    result = dict(sorted(dict_elem.items(), key = itemgetter(1), reverse = True)[:n]) 
    return result

print(get_top_n(lda_model, 3))

In [ ]:
header = ['Page_ID', 'Topics', 'Co-Req ID', '']
data = ['...', lda_model.print_topics(), ]

filename = 'Volumes/CIRCUIT_HD2/CRIMSON/Topic_Modeling_Practice/prototype_topics.csv'

with open(filename, 'w', encoding='UTF8') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write the data
    writer.writerow(data)
